### Doc Writer Ai Agent

In [73]:
# Import necessary libraries
import os
import time
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage
from typing import Optional, TypedDict


In [74]:
# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Check and print status
print("API key loaded successfully!" if GOOGLE_API_KEY else "API key not found!")

API key loaded successfully!


In [75]:
# make sure to set the GOOGLE_API_KEY in your .env file
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    temperature=0.5,
    google_api_key=GOOGLE_API_KEY
)
# making sure llm is working add a print statement
print("LLM initialized successfully!")

LLM initialized successfully!


In [76]:
# defining the AgentState class
class AgentState(TypedDict):
    """State of the agent."""
    topic: str  # Topic of the document/email
    draft: Optional[str]  # Current draft generated/refined
    feedback: Optional[str]  # Feedback from human (if any)
    is_satisfied: Optional[bool]  # Whether human is happy with the draft

In [77]:
def ai_node(s: AgentState) -> AgentState:
    """
    Generates or refines a document draft based on the topic or existing draft.

    This node checks if a draft already exists in the state. If not, it prompts
    the LLM to generate a new document based on the provided topic. If a draft
    exists, it asks the LLM to refine the current draft. The resulting content
    is stored in the 'draft' key of the state.

    Args:
        s (AgentState): The current state of the agent containing at least a 'topic',
                        and optionally a 'draft'.

    Returns:
        AgentState: The updated state with a new or refined 'draft'.
    """
    if s['draft'] is None:
        prompt = f"Write a document about {s['topic']}."
    else:
        prompt = f"Refine the following document: {s['draft']}"

    # Simulate typing/processing
    print("🧠 Generating response", end="", flush=True)
    for _ in range(3):
        time.sleep(0.5)
        print(".", end="", flush=True)
    print("\n")

    llm.invoke([HumanMessage(content=prompt)])

    s['draft'] = response.content
    return s

In [78]:
def show_to_human(s: AgentState) -> AgentState:
    """
    Displays the generated draft to the user and collects satisfaction feedback.

    This node prints the current 'draft' to the console and prompts the user with
    a yes/no question after a short pause.
    """
    print(f"\n📝 Draft:\n{s['draft']}\n")

    # Optional delay before asking
    print("Processing complete. Preparing to ask for your feedback...")
    time.sleep(10)  # wait for 2.5 seconds

    feedback = input("Are you satisfied with the draft? (yes/no): ").strip().lower()

    s['is_satisfied'] = True if feedback == "yes" else False
    return s


In [79]:
def take_feedback(s: AgentState) -> AgentState:
    """
    Prompts the human user for feedback if the draft is unsatisfactory.

    If 'is_satisfied' is False, this node asks the user to provide feedback
    on how the draft can be improved. The feedback is stored in the 'feedback'
    key of the state. If the user is satisfied, any existing feedback is cleared.

    Args:
        s (AgentState): The current agent state, including satisfaction status.

    Returns:
        AgentState: The updated state with 'feedback' set to user input or None.
    """
    if not s['is_satisfied']:
        feedback = input("Please provide your feedback on the draft: ")
        s['feedback'] = feedback
    else:
        s['feedback'] = None  # Clear feedback if satisfied

    print(f"Feedback recorded: {s['feedback']}")
    return s


In [80]:
def refine_node(s: AgentState) -> AgentState:
    """
    Refines the current draft based on human feedback using the LLM.
    """
    if s['feedback'] is not None:
        prompt = (
            f"Please revise the following draft to better address the feedback provided. "
            f"Be clear, concise, and accurate.\n\n"
            f"Feedback: {s['feedback']}\n\n"
            f"Original Draft:\n{s['draft']}"
        )

        print("🧠 Refining draft", end="", flush=True)
        for _ in range(3):
            time.sleep(0.5)
            print(".", end="", flush=True)
        print("\n")

        response = llm.invoke(prompt)  # ✅ Pass as plain string
        s['draft'] = f"Refined Draft:\n{response.content}"
    else:
        print("No feedback provided, skipping refinement.")

    return s


In [81]:
import datetime
import os


def save_draft(s: AgentState):
    """
    Saves the final approved draft and optional feedback to a text file.

    If the draft has been approved (is_satisfied is True), this node creates a
    timestamped .txt file containing the topic, draft, and feedback. Otherwise,
    it prints a message that the draft was not saved.

    Args:
        s (AgentState): The final state containing 'topic', 'draft', and 'feedback'.
    """
    if s.get('is_satisfied') == True:
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"final_draft_{timestamp}.txt"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(f"Topic: {s.get('topic', 'Unknown')}\n\n")
            f.write(f"Draft:\n{s.get('draft', 'No draft')}\n\n")
            f.write(f"Feedback: {s.get('feedback', 'No feedback provided')}\n")
        print(f"✅ Draft saved as {filename} in {os.getcwd()}")
    else:
        print("❌ Draft not saved as it was not approved by the user.")


In [82]:
from langgraph.graph import StateGraph

workflow = StateGraph(AgentState)

# Add all your nodes
workflow.add_node("ai_draft", ai_node)
workflow.add_node("show_to_human", show_to_human)
workflow.add_node("take_feedback", take_feedback)
workflow.add_node("refine_draft", refine_node)
workflow.add_node("save_draft", save_draft)

# Add edges
workflow.set_entry_point("ai_draft")
workflow.add_edge("ai_draft", "show_to_human")


# Conditional branch based on satisfaction
def condition(state: AgentState) -> str:
    return "save_draft" if state["is_satisfied"] else "take_feedback"


workflow.add_conditional_edges(
    "show_to_human",
    condition,
    {
        "save_draft": "save_draft",
        "take_feedback": "take_feedback"
    }
)

# Continue loop if not satisfied
workflow.add_edge("take_feedback", "refine_draft")
workflow.add_edge("refine_draft", "ai_draft")

# Compile the graph
app = workflow.compile()


In [ ]:
initial_state = {
    "topic": "The impact of AI Healthcare",
    "draft": None,
    "feedback": None,
    "is_satisfied": None
}
final_state = app.invoke(initial_state)